# Transforma dataset de scrapy sneaky 

Nueva version en harvesterpy/scripts

In [1]:
import glob
import gzip
import json
import re

In [2]:
class RegexDict(dict):

    def get_dict_matching(self, regex):
        return {key: self[key] for key in self.keys() if re.match(regex, key)}
        
    def get_matching(self, regex, value_field=None):
        results = []
        for key in self.keys():
            if re.match(regex, key):
                value = self[key]
                if value_field:
                    value[value_field] = key
                results.append(value)
        return results

In [3]:
day = "20230802"
categories = {}
subcategories = {}

files = f"harvester/occ/{day}/sneaky_spider*.jsonl"
num_files = 0
records = 0
num_exceptions = 0
total_lines = 0
exceptions = []

with open(f"occ-{day}.jsonl", "w") as fo:
    for infile in glob.glob(files):
        num_files += 1
        print(".", end="")
#        print(infile)
#        with gzip.open(infile) as fin:
        with open(infile) as fin:
            line_num = 0
            for line in fin:
                try:
                    #line = fin.readline()
                    j = json.loads(line)
                    jobp = json.loads(j["jobposting"])
                    jobp = jobp["props"]["pageProps"]["initialApolloState"]
                    jobext = RegexDict(jobp)
                    job = jobext.get_matching(r"Job:.*", value_field="Job:value")[0]
                    job['scraped_at'] = j['scraped_at']
                    job['scraped_url'] = j['url']
                    job['scraped_uuid'] = j['uuid']
                    job['scraped_record_version'] = j['version']
                    job['scraped_identifier'] = j['identifier']
                    job['scraped_crawler'] = j['crawler']
                    scraper_uuid = re.match(r"^.*-.*-(.*)-rv.*-b.*.jsonl$", infile)
                    job['scraped_by_uuid'] = scraper_uuid.group(1)
                    
                    fo.write(json.dumps(job))
                    fo.write("\n")
                    
                    cat = jobext.get_matching("JobCategory:.*")[0]
                    subcat = jobext.get_matching("JobSubcategory:.*")[0]
                    categories[cat['id']] = cat
                    subcategories[subcat['id']] = subcat
                    records += 1
                except Exception as ex:
#                    print(f"line {line_num}: {ex}")
                    exceptions.append({'file': infile, 'line': line_num, 'exception': str(ex)})
                    num_exceptions += 1
                finally:
                    line_num += 1
                    total_lines += 1

...........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [4]:
print(f"\nFiles: {num_files}\tTotal lines: {total_lines} \tRecords: {records}\tExceptions: {num_exceptions}")


Files: 539	Total lines: 132565 	Records: 132275	Exceptions: 290


In [5]:
with open(f"occ-{day}-exceptions.jsonl", "w") as fo:
    for ex in exceptions:
        fo.write(json.dumps(ex))
        fo.write("\n")

In [6]:
with open(f"occ-{day}-categories.json", "w") as fo:
    fo.write(json.dumps([c for c in categories.values()]))

with open(f"occ-{day}-subcategories.json", "w") as fo:
    fo.write(json.dumps([c for c in subcategories.values()]))
             
len(categories), len(subcategories)

(24, 325)

In [7]:
jobext = RegexDict(jobp)
jobext.get_matching(r"Job:.*", value_field="Job:value")[0]

{'__typename': 'Job',
 'id': '17091221',
 'url': '/empleo/oferta/17091221-vendedor-piso-de-ventas-elektra-valle-verde?rank=1&page=1&sessionid=&userid=&uuid=5f74e33d-a0cf-42e2-8189-92e493c924ce&origin=unknown&type=3&ai=false&ais=&showseo=true&returnURL=%2Fempleos-en-mexico-y-el-mundo%3Fsessionid%3D%26userid%3D%231',
 'title': 'Vendedor Piso De Ventas Elektra Valle Verde',
 'description': 'UNETE A GRUPO ELEKTRA!!     SOLICITAMOS VENDEDOR , SUCURSAL VALLE VERDE (Sobre Av Abraham Lincoln, frente a Plaza Adana)     REQUISITOS:  *PROACTIVO, DINAMICO     *GUSTO POR LAS VENTAS     *EXPERIENCI ...',
 'jobType': 'PREMIUM',
 'salary': {'__typename': 'JobSalary',
  'show': True,
  'from': 8000,
  'to': 12000,
  'time': 0,
  'performanceCompensation': 1,
  'variableCompensation': 0},
 'location': {'__typename': 'JobLocation',
  'description': 'Monterrey, Nuevo León',
  'locations': [{'__typename': 'JobLocationData',
    'city': {'__typename': 'CityLocation',
     'description': 'Monterrey',
     'j

In [8]:
cat = jobext.get_matching("JobCategory:.*")[0]
subcat = jobext.get_matching("JobSubcategory:.*")[0]
cat, subcat

({'__typename': 'JobCategory',
  'description': 'Ventas',
  'id': '19',
  'url': 'empleos/trabajo-en-ventas/',
  'rel': None},
 {'__typename': 'JobSubcategory',
  'id': '275',
  'description': 'Administración de ventas',
  'url': 'empleos/trabajo-en-ventas-administracion-de-ventas/'})